# GLiNER2 한국어 학습 (Colab Pro GPU)

이 노트북은 `klue/roberta-base`를 사용하여 GLiNER2를 한국어 데이터로 학습합니다.

## 1. GPU 확인

In [ ]:
!nvidia-smi

## 2. Google Drive 마운트 (선택사항)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. 프로젝트 업로드 또는 클론

In [ ]:
# 방법 1: GitHub에서 클론 (GitHub에 올려둔 경우)
# !git clone https://github.com/your-username/your-repo.git
# %cd your-repo

# 방법 2: 로컬 파일 업로드
# 왼쪽 파일 탭에서 프로젝트 폴더를 드래그 앤 드롭

# 방법 3: Google Drive에서 복사 (Drive 마운트 후)
# !cp -r /content/drive/MyDrive/graph_lang /content/
# %cd graph_lang

print("현재 디렉토리:")
!pwd
!ls -la

## 4. 필수 패키지 설치

In [ ]:
# PyTorch 및 transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers>=4.30.0
!pip install accelerate

# 한국어 토크나이저
!pip install mecab-python3
!pip install python-mecab-ko

# 기타 필수 패키지
!pip install safetensors
!pip install tqdm
!pip install numpy
!pip install huggingface_hub

# GLiNER 관련 (gliner 패키지에서 SpanRepLayer 사용)
!pip install gliner

## 5. MeCab 사전 설치 (한국어)

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

## 6. 데이터 확인

In [ ]:
import json

# 학습 데이터 확인
with open('data/train_ko_2k.jsonl', 'r', encoding='utf-8') as f:
    train_samples = [json.loads(line) for line in f]

print(f"학습 데이터: {len(train_samples)}개")
print("\n첫 번째 샘플:")
print(json.dumps(train_samples[0], ensure_ascii=False, indent=2))

# 검증 데이터 확인
try:
    with open('data/val.jsonl', 'r', encoding='utf-8') as f:
        val_samples = [json.loads(line) for line in f]
    print(f"\n검증 데이터: {len(val_samples)}개")
except FileNotFoundError:
    print("\n검증 데이터 없음")

## 7. 학습 설정

In [ ]:
# 학습 하이퍼파라미터
TRAIN_DATA = "data/train_ko_2k.jsonl"
VAL_DATA = "data/val.jsonl"  # 없으면 None으로 설정
OUTPUT_DIR = "./output_ko"
BASE_MODEL = "klue/roberta-base"  # 한국어 사전학습 모델

NUM_EPOCHS = 20
BATCH_SIZE = 16  # Colab Pro GPU에 맞게 조정 (V100: 16-32, A100: 32-64)
GRADIENT_ACCUMULATION = 1  # 메모리 부족시 2 또는 4로 증가
ENCODER_LR = 1e-5
TASK_LR = 5e-4
SAVE_STEPS = 500
EVAL_STEPS = 500
LOGGING_STEPS = 100

print(f"학습 데이터: {TRAIN_DATA}")
print(f"검증 데이터: {VAL_DATA}")
print(f"출력 디렉토리: {OUTPUT_DIR}")
print(f"베이스 모델: {BASE_MODEL}")
print(f"에포크: {NUM_EPOCHS}")
print(f"배치 크기: {BATCH_SIZE}")
print(f"Effective 배치 크기: {BATCH_SIZE * GRADIENT_ACCUMULATION}")

## 8. 모델 및 Trainer 초기화

In [ ]:
from gliner2.model import Extractor, ExtractorConfig
from gliner2.training.trainer import GLiNER2Trainer, TrainingConfig

# 모델 초기화 (klue/roberta-base로 새로 만들기)
print(f"모델 초기화: {BASE_MODEL}")
config = ExtractorConfig(
    model_name=BASE_MODEL,
    max_width=8,
    counting_layer="count_lstm",
    token_pooling="first"
)
model = Extractor(config)

print("\n모델 파라미터:")
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total: {total_params:,}")
print(f"Trainable: {trainable_params:,}")

## 9. Training Config 설정

In [ ]:
training_config = TrainingConfig(
    output_dir=OUTPUT_DIR,
    experiment_name="gliner2-ko",
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    encoder_lr=ENCODER_LR,
    task_lr=TASK_LR,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_total_limit=3,
    save_best=True,
    logging_steps=LOGGING_STEPS,
    fp16=True,  # Mixed precision for faster training
    num_workers=2,
    seed=42,
    validate_data=True,
)

print("Training Config:")
print(f"  Epochs: {training_config.num_epochs}")
print(f"  Batch size: {training_config.batch_size}")
print(f"  Effective batch size: {training_config.effective_batch_size}")
print(f"  Encoder LR: {training_config.encoder_lr}")
print(f"  Task LR: {training_config.task_lr}")
print(f"  FP16: {training_config.fp16}")

## 10. Trainer 생성 및 학습 시작

In [ ]:
import os

# Trainer 생성
trainer = GLiNER2Trainer(
    model=model,
    config=training_config,
)

# 학습 시작
print("\n" + "="*60)
print("학습 시작!")
print("="*60 + "\n")

# VAL_DATA가 없으면 None으로 설정
val_data = VAL_DATA if os.path.exists(VAL_DATA) else None

results = trainer.train(
    train_data=TRAIN_DATA,
    eval_data=val_data,
)

print("\n" + "="*60)
print("학습 완료!")
print("="*60)
print(f"Total steps: {results['total_steps']}")
print(f"Total epochs: {results['total_epochs']}")
print(f"Total time: {results['total_time_seconds']:.2f}s")
print(f"Samples/sec: {results['samples_per_second']:.2f}")
print(f"Best metric: {results['best_metric']:.4f}")

## 11. 학습된 모델 테스트

In [ ]:
from gliner2 import GLiNER2

# 최고 성능 모델 로드
best_model_path = f"{OUTPUT_DIR}/best"
model = GLiNER2.from_pretrained(best_model_path)

# 테스트
test_text = "삼성전자는 서울에 본사를 두고 있으며, 이재용 부회장이 경영하고 있다."
entities = ["회사명", "지역", "인물"]

results = model.extract_entities(test_text, entities)

print(f"텍스트: {test_text}")
print(f"\n추출된 엔티티:")
for entity in results:
    print(f"  - {entity['label']}: {entity['text']} (score: {entity['score']:.3f})")

## 12. 모델 다운로드 (로컬로 가져가기)

In [ ]:
# 방법 1: Google Drive에 저장
!cp -r {OUTPUT_DIR}/best /content/drive/MyDrive/gliner2_ko_best
print("모델이 Google Drive에 저장되었습니다: /content/drive/MyDrive/gliner2_ko_best")

# 방법 2: ZIP으로 압축 후 다운로드
!zip -r gliner2_ko_best.zip {OUTPUT_DIR}/best
print("\nZIP 파일 생성 완료: gliner2_ko_best.zip")
print("왼쪽 파일 탭에서 다운로드하세요.")

# 방법 3: Colab 파일 다운로드
from google.colab import files
# files.download('gliner2_ko_best.zip')  # 주석 해제하면 자동 다운로드

## 13. Hugging Face Hub에 업로드 (선택사항)

In [ ]:
# Hugging Face 토큰으로 로그인
# from huggingface_hub import login
# login()

# 모델 업로드
# model.push_to_hub("your-username/gliner2-ko", private=True)
# print("모델이 Hugging Face Hub에 업로드되었습니다!")